In [6]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os, cv2
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.datasets import load_files
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications import *

train_path = './train'
validate_path = './val'
test_path = './test'


In [7]:
dim = 299
epochs = 50
learning_rate = 0.0001
batch_size = 16


train_num = 3790
validation_num = 480

In [8]:

weights = os.path.join('', 'weights_vgg16.h5')

datagen = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip=True,
                             width_shift_range=0.1,
                             height_shift_range=0.1)


In [9]:
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(dim, dim),
    batch_size=batch_size,
    class_mode='categorical')

validate_generator = datagen.flow_from_directory(
    validate_path,
    target_size=(dim, dim),
    batch_size=batch_size,
    class_mode='categorical')

Found 3790 images belonging to 12 classes.
Found 480 images belonging to 12 classes.


In [10]:

callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=0),
             ModelCheckpoint(weights, monitor='val_loss', save_best_only=True, verbose=1),
             ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=0, mode='auto', epsilon=0.0001,
                               cooldown=0, min_lr=0)]

In [11]:

base_model = VGG16(input_shape=(dim, dim, 3), include_top=False, weights='imagenet', pooling='avg')
x = base_model.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=learning_rate), metrics=['accuracy'])


In [12]:

model.fit_generator(train_generator,
                    steps_per_epoch=train_num / batch_size,
                    validation_data=validate_generator,
                    validation_steps=validation_num/ batch_size,
                    callbacks=callbacks,
                    epochs=epochs,
                    verbose=1)

Epoch 1/50
237/236 [==============================] - 448s - loss: 2.4801 - acc: 0.1282 - val_loss: 2.4958 - val_acc: 0.0833
Epoch 2/50
237/236 [==============================] - 441s - loss: 2.2712 - acc: 0.2043 - val_loss: 2.2523 - val_acc: 0.2479
Epoch 3/50
237/236 [==============================] - 439s - loss: 1.7140 - acc: 0.4444 - val_loss: 2.1791 - val_acc: 0.2562
Epoch 4/50
237/236 [==============================] - 438s - loss: 1.1701 - acc: 0.6121 - val_loss: 0.8743 - val_acc: 0.6813
Epoch 5/50
237/236 [==============================] - 437s - loss: 0.7340 - acc: 0.7602 - val_loss: 0.5871 - val_acc: 0.7688
Epoch 6/50
237/236 [==============================] - 437s - loss: 0.5683 - acc: 0.8114 - val_loss: 0.5571 - val_acc: 0.8063
Epoch 7/50
237/236 [==============================] - 436s - loss: 0.4865 - acc: 0.8381 - val_loss: 0.4998 - val_acc: 0.8187
Epoch 8/50
237/236 [==============================] - 436s - loss: 0.4239 - acc: 0.8562 - val_loss: 0.3972 - val_acc: 0.8646


237/236 [==============================] - 431s - loss: 0.1142 - acc: 0.9641 - val_loss: 0.5219 - val_acc: 0.8750
Epoch 32/50
237/236 [==============================] - 431s - loss: 0.1425 - acc: 0.9566 - val_loss: 0.3005 - val_acc: 0.9354
Epoch 33/50
237/236 [==============================] - 432s - loss: 0.1664 - acc: 0.9536 - val_loss: 0.3468 - val_acc: 0.8833
Epoch 34/50
237/236 [==============================] - 431s - loss: 0.0918 - acc: 0.9713 - val_loss: 0.2034 - val_acc: 0.9333
Epoch 35/50
237/236 [==============================] - 431s - loss: 0.0644 - acc: 0.9807 - val_loss: 0.2174 - val_acc: 0.9333


In [13]:

weights = os.path.join('', 'weights_vgg16.h5')
test_datagen = ImageDataGenerator()       
generator_test = test_datagen.flow_from_directory(
    test_path,
    target_size=(dim, dim),
    batch_size=batch_size,
    class_mode=None,  
    shuffle=False)  


Found 480 images belonging to 12 classes.


In [14]:

filenames = generator_test.filenames
nb_samples = len(filenames)
print(nb_samples)
prediction_test = model.predict_generator(generator_test, verbose=1,steps =30)

480
30/30 [==============================] - 16s    


In [15]:
print("done")


done


In [16]:
prediction_list = []
for i in range(len(prediction_test)):
    result = np.argmax(prediction_test[i])
    prediction_list.append(result)

count = 0

for i in range(len(prediction_list)):
    if prediction_list[i] == generator_test.classes[i]:
        
        count += 1
print(count/480)

480
480
0.9604166666666667
